In [ ]:
import numpy as np
import time
from numpy import ones,vstack
from numpy.linalg import lstsq

import math
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from random import random, randint, uniform

import os
import subprocess
import shutil

from statistics import median

In [ ]:
def parse_trace(filename):
    with open(filename, "r") as f:
        lines = f.readlines()

    trajectory = []
    for i in range(0, len(lines), 2):
        line = lines[i]
        state = {}
        for tup in line.strip().split(" "):
            variable = tup.split("=")[0][1:-1]
            value = round(float(tup.split("=")[1]),3)
            state[variable] = value

        trajectory += [state]
    return trajectory

In [ ]:
def parse_problem(problem_file):

    x_obs = []
    t_obs = []
    on = False

    with open(problem_file, "r") as f:
        for line in f.readlines():
            if "(x)" in line:
                init_x = float(line.strip().replace("(", "").replace(")", "").split(" ")[2])
            elif "(on)" in line:
                on = True
            elif "(time_obs" in line:
                t_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]
            elif "(x_obs" in line:
                x_obs += [float(line.strip().replace("(", "").replace(")", "").split(" ")[3])]


    observations = list(zip(x_obs, t_obs))
    
    return observations, init_x, on

In [ ]:
def generate_observation(state, k):
    observation = (state[0],state[1],state[2])
    
    return observation

In [ ]:
def generate_decoding_domain(observations, error=0.05, time_precision=0.05):
    
    pddl_str = "(define (domain thermostat)\n"
    pddl_str += "\n"
    
    pddl_str += "(:requirements :fluents :typing :time )\n"
    pddl_str += "\n"
    
    # Predicates
    pddl_str += "(:predicates\n"
    pddl_str += "\t;HS locations\n"
    pddl_str += "\t(on)\n"
    pddl_str += "\t(off)\n"
    pddl_str += "\n"    
    pddl_str += "\t;monitor locations\n"
    for i in range(len(observations)+1):
        pddl_str += "\t(monitor_{})\n".format(i)
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # Functions
    pddl_str += "(:functions\n"
    pddl_str += "\t;HS variables\n"
    pddl_str += "\t(x)  ;; temperature\n"
    pddl_str += "\n"
    
    pddl_str += "\t;monitor variables\n"
    pddl_str += "\t(running_time)  ;; time\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # HS Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-on\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(on)\n"
    pddl_str += "\t\t(<= (x) 22)\n"  
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x) (* #t 0.8))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    pddl_str += "(:process flow-off\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(off)\n"
    pddl_str += "\t\t(>= (x) 18)\n"  
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(increase (x) (* #t -0.2))\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    # HS Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; HS Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    pddl_str += "(:action turn-on\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(off)\n"
    pddl_str += "\t\t(< x 19)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (off))\n"
    pddl_str += "\t\t(on)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    pddl_str += "(:action turn-off\n"
    pddl_str += "\t:parameters ()\n"
    pddl_str += "\t:precondition (and\n"
    pddl_str += "\t\t(on)\n"
    pddl_str += "\t\t(> x 21)\n"
    pddl_str += "\t)\n"
    pddl_str += "\t:effect (and\n"
    pddl_str += "\t\t(not (on))\n"
    pddl_str += "\t\t(off)\n"
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += "\n"
    
    
    # Monitor Continuous transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Continuous transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
    
    
    for i in range(len(observations)+1):
        pddl_str += "(:process flow-monitor_{}\n".format(i)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(increase (running_time) (* #t 1.0))\n"
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
        
        
    # Monitor Discrete transitions
    
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += ";;; Monitor Discrete transitions\n"
    pddl_str += ";;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;\n"
    pddl_str += "\n"
        
        
    for i in range(len(observations)):
        obs = observations[i]
        pddl_str += "(:event validate_{}\n".format(i+1)
        pddl_str += "\t:parameters ()\n"
        pddl_str += "\t:precondition (and\n"
        pddl_str += "\t\t(monitor_{})\n".format(i)
        pddl_str += "\t\t(<= {} (running_time)) ; t_{} - 0.05\n".format(obs[1] - time_precision, i+1)
        pddl_str += "\t\t(> {} (running_time)) ; t_{} + 0.05\n".format(obs[1] + time_precision, i+1)
        pddl_str += "\t\t(<= {} (x)) ; \phi_{}\n".format(obs[0] - error, i+1)
        pddl_str += "\t\t(>= {} (x)) ; \phi_{}\n".format(obs[0] + error, i+1)
        pddl_str += "\t)\n"
        pddl_str += "\t:effect (and\n"
        pddl_str += "\t\t(not (monitor_{}))\n".format(i)
        pddl_str += "\t\t(monitor_{})\n".format(i+1)
        pddl_str += "\t)\n"
        pddl_str += ")\n"
        pddl_str += "\n"
        
    pddl_str += ")"
    
    return pddl_str
    

In [ ]:
def generate_decoding_problem(observations, starting_temp=19):
    
    pddl_str = "(define (problem problem_name) (:domain decoding)\n"
    pddl_str += "\n"

    # Initial state
    pddl_str += "(:init\n"

    pddl_str += "\t(= (x) {v})\n".format(v=starting_temp)
    if starting_temp <= 20:
        pddl_str += "\t(off)\n"
    else:
        pddl_str += "\t(on)\n"
    pddl_str += "\t(= (running_time) 0)\n"
    pddl_str += "\t(monitor_0)\n"
    pddl_str += "\n"

    pddl_str += ")\n"
    
    pddl_str += "(:goal\n"
    pddl_str += "\t(and\n"
    pddl_str += "\t\t(monitor_{})\n".format(len(observations))
    pddl_str += "\t)\n"
    pddl_str += ")\n"
    pddl_str += ")\n"
    
    return pddl_str
    

In [ ]:
num_instances = 10
horizons = [2, 4, 6, 8, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

dfolder = "benchmarks/thermostat/UPM_e"
if os.path.exists(dfolder):
    shutil.rmtree(dfolder)
os.makedirs(dfolder)


for pnum in range(num_instances):
    for h in horizons:
        dname = 'domain_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        pname = 'inst_{pnum}_{h}.pddl'.format(pnum=pnum, h=h)
        
        observations, init_x, on = parse_problem("benchmarks/thermostat/generation/inst_{}_{}_5_100.pddl".format(pnum, h))
        observations = observations[1:]
        
        
        domain_str = generate_decoding_domain(observations)
        with open(dfolder + '/' + dname, 'w') as f:
            f.write(domain_str)
            
        problem_str = generate_decoding_problem(observations, starting_temp=init_x)
        with open(dfolder + '/' + pname, 'w') as f:
            f.write(problem_str)

In [ ]:
times = {}
instances = list(range(num_instances))

cmd0 = "cd {dfolder}".format(dfolder=dfolder)
os.system(cmd0)

time_budget = 300

for h in horizons:
    h_times = []
    working_instances = []
    print(instances)
    for pnum in instances:  
        
        try:
            dname = 'domain_{pnum}_{h}'.format(pnum=pnum, h=h)
            pname = 'inst_{pnum}_{h}'.format(pnum=pnum, h=h)

            cmd1 = "cd {dfolder}; upmc {dname}.pddl {pname}.pddl --custom 0.01 10 4 --clean --force".format(dfolder=dfolder, dname=dname, pname=pname)
            
            t = time.time()
            os.system(cmd1)
            elapsed = time.time() - t
            
            cmd2 = "cd {dfolder}; timeout {timeout} ./{dname}_planner -m 8000".format(dfolder=dfolder, dname=dname, timeout=int(300-elapsed))
            
            os.system(cmd2)
            elapsed = time.time() - t
            
            os.system("cd {dfolder}; rm *.transitions *.properties *.startstates").format(dfolder=dfolder)
            
            with open ("{dfolder}/{pname}_plan.pddl".format(dfolder=dfolder, pname=pname), "r") as f:
                content = f.read()
            assert content != ""

            h_times += [elapsed]
            working_instances += [pnum]
        except:
            print("instance {} failed".format(pnum))
        
    times[h] = h_times
    print(h, h_times)
    
    instances = working_instances
    
with open("{}/upm_e".format(dfolder), "w") as f:
    for k,h_times in times.items():
        f.write(" ".join(map(str, h_times)) + "\n")
